In [8]:
import sys
sys.path.append("..")

In [9]:
from src.datasets import HateXplainRace

In [10]:
hate = HateXplainRace('../data/HateXplain')

In [11]:
import faiss
from sentence_transformers import SentenceTransformer

embedding = SentenceTransformer('all-mpnet-base-v2')
hate = HateXplainRace("../data/HateXplain/")
train_df, test_df, overall_demographics = hate.create_prompts()

train_vectors = embedding.encode(train_df['prompts'].tolist(), batch_size = 32, show_progress_bar=True)
test_vectors = embedding.encode(test_df['prompts'].tolist(), batch_size = 32, show_progress_bar=True)
faiss.normalize_L2(train_vectors)

faiss.normalize_L2(test_vectors)
vector_dim = train_vectors.shape[1]

index = faiss.IndexFlatIP(vector_dim)

index.add(train_vectors)

distances, neighbors = index.search(test_vectors, 5)

Batches: 100%|██████████| 22/22 [00:00<00:00, 41.93it/s]


In [12]:
differences_in_demographic = []

for i in range(len(neighbors)):
    demographics_train = train_df.iloc[neighbors[i]]['filtered_demographics']
    demographics_test = test_df.iloc[i]['filtered_demographics']
    differences_in_demographic.append(len(demographics_train[demographics_train != demographics_test]))

In [13]:
import numpy as np

In [14]:
np.array(differences_in_demographic).mean()

1.353891336270191